In [4]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from transformers import DonutProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests
import torch
from MMObject.utls import *


In [5]:
# # processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
# # model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")
# from transformers import AutoProcessor, VisionEncoderDecoderModel
# from PIL import Image
# import requests
# import torch

# # Use the general AutoProcessor to load Donut's processor
# model_name = "naver-clova-ix/donut-base"
# processor = AutoProcessor.from_pretrained(model_name)
# model = VisionEncoderDecoderModel.from_pretrained(model_name)
# # model = VisionEncoderDecoderModel.from_pretrained(model_name)
# model.eval() 
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True)
model = AutoModel.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True, low_cpu_mem_usage=True, device_map='cuda', use_safetensors=True, pad_token_id=tokenizer.eos_token_id)
model = model.eval().cuda()


split = "test"

path_dict = {"val":"/home/ubuntu/data/coco/val2017/",
             "test":"/home/ubuntu/data/coco/test2017/",
             "train":"/home/ubuntu/data/coco/train2017/"}

val_aokvqa, coco_val_caption, coco_id_filename = prepare_dataset(split = split)

ocr_res = {}
for i in trange(len(val_aokvqa)):
    case = val_aokvqa[i]
    img_id = case['image_id']
    base_path = path_dict[split]
    img_file = coco_id_filename[img_id]
    img_path = base_path + img_file  
    res = model.chat(tokenizer, img_path, ocr_type='ocr')
    ocr_res[img_id] = res


In [29]:
# save the json file
ocr_res_path = f"/home/ubuntu/data/aokvqa/ocr_res_{split}.json"
with open(ocr_res_path, 'w') as f:
    json.dump(ocr_res, f)

In [9]:
# pixel_values = processor(images=image, return_tensors="pt").pixel_values

# # Generate predictions
# with torch.no_grad():
#     generated_ids = model.generate(pixel_values)

# # Decode the generated text
# generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

# # Output the recognized text
# print("Recognized Text:", generated_text)

Recognized Text: 0 1


In [1]:
import re

In [2]:
text = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

<|image|>
You task is to select one of the four following options based on the image and the question. Specifically, you need to output 0 or 1 or 2 or 3 as well as rationales of why you chose that option.
The rationales should also include in curly braces the answer to the question.

Here are some examples that you can follow:
Question: What is in the motorcyclist's mouth?
Choices: 0. toothpick 1. food 2. popsicle stick 3. cigarette
Rationale: Looking at the picture, we can see a man with white shirt riding a motorcycle. Now as we find the motorcyclist in the question, we can see that he has a cigarette in his mouth.

Question: Which number birthday is probably being celebrated?
Choices: 0. one 1. ten 2. nine 3. thirty
Rationale: From the question, we know that someone is celebrating for the birthday. Looking at the picture, we can see two things on the table top. One is a grey bear which is likely to be a cake, the other is a cake with several candles on it. By looking at the purple cake, we can see it writes number thirty on it.

Question: What best describes the pool of water?
Choices: 0. frozen 1. fresh 2. dirty 3. boiling
Rationale: Looking at the picture, we can see a tree in the middle. Behind the tree, we can see several giraffes. On the bottom of the picture, we can see a pool of water. This refers to the pool mentioned in the question. The pool is dark brown and it is brown and surrounded with mud. So the pool is dirty.

Now, it's your turn. Again, remember to put your answer in curly braces. Here is the question you need to answer.
Here is the OCR result of the image. You can refer to this information to answer the question. Remember the OCR result is not always accurate.
       
 OCR result: COUNTRY
 Now, here is the question you need to answer.
    Question: What type of event might this be?
Choices: 0. art show 1. car show 2. museum opening 3. race
Now Please answer the following question based on the image and the OCR result. You should first output Rational and then the answer.
 Now, please output the rational step by step.
 Rational:The image shows a car covered in various objects, including what appears to be a doll's head, a toothbrush, and a box of cereal. The car is parked in a crowded area, with people milling about in the background. The presence of the doll's head and the other objects on the car suggests that this may be some kind of art installation or a display of unusual items.

Given the context of the image, it is likely that this is a car show, where people showcase their unique or decorated vehicles. The fact that the car is covered in a variety of objects and the crowd in the background suggests an event where people are gathered to admire and appreciate these vehicles.

Therefore, the answer is:

1. car show<|eot_id|>
 Now, please output the answer using 0 or 1 or 2 or 3.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

1<|eot_id|>"""

In [4]:
print(re.sub(r'<\|.*\|>', '', text))




You task is to select one of the four following options based on the image and the question. Specifically, you need to output 0 or 1 or 2 or 3 as well as rationales of why you chose that option.
The rationales should also include in curly braces the answer to the question.

Here are some examples that you can follow:
Question: What is in the motorcyclist's mouth?
Choices: 0. toothpick 1. food 2. popsicle stick 3. cigarette
Rationale: Looking at the picture, we can see a man with white shirt riding a motorcycle. Now as we find the motorcyclist in the question, we can see that he has a cigarette in his mouth.

Question: Which number birthday is probably being celebrated?
Choices: 0. one 1. ten 2. nine 3. thirty
Rationale: From the question, we know that someone is celebrating for the birthday. Looking at the picture, we can see two things on the table top. One is a grey bear which is likely to be a cake, the other is a cake with several candles on it. By looking at the purple cake, we